## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


**534. Game Play Analysis III (Medium)**

**Table: Activity**

| Column Name  | Type    |
|--------------|---------|
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |

(player_id, event_date) is the primary key (column with unique values) of this table.
This table shows the activity of players of some games.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on someday using some device.
 
**Write a solution to report for each player and date, how many games played so far by the player. That is, the total number of games played by the player until that date. Check the example for clarity.**

Return the result table in any order.

The result format is in the following example.

**Example 1:**

**Input:** 

**Activity table:**

| player_id | device_id | event_date | games_played |
|-----------|-----------|------------|--------------|
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-05-02 | 6            |
| 1         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |

**Output:** 
| player_id | event_date | games_played_so_far |
|-----------|------------|---------------------|
| 1         | 2016-03-01 | 5                   |
| 1         | 2016-05-02 | 11                  |
| 1         | 2017-06-25 | 12                  |
| 3         | 2016-03-02 | 0                   |
| 3         | 2018-07-03 | 5                   |

**Explanation:** 
- For the player with id 1, 5 + 6 = 11 games played by 2016-05-02, and 5 + 6 + 1 = 12 games played by 2017-06-25.
- For the player with id 3, 0 + 5 = 5 games played by 2018-07-03.

**Note** that for each player we only care about the days when the player logged in.

In [0]:
activity_data_534 = [
    (1, 2, '2016-03-01', 5),
    (1, 2, '2016-05-02', 6),
    (1, 3, '2017-06-25', 1),
    (3, 1, '2016-03-02', 0),
    (3, 4, '2018-07-03', 5)
]

activity_columns_534 = ['player_id', 'device_id', 'event_date', 'games_played']
activity_df_534 = spark.createDataFrame(activity_data_534, activity_columns_534)
activity_df_534.show()

+---------+---------+----------+------------+
|player_id|device_id|event_date|games_played|
+---------+---------+----------+------------+
|        1|        2|2016-03-01|           5|
|        1|        2|2016-05-02|           6|
|        1|        3|2017-06-25|           1|
|        3|        1|2016-03-02|           0|
|        3|        4|2018-07-03|           5|
+---------+---------+----------+------------+



In [0]:
windowSpec = Window.orderBy("event_date")

activity_df_534\
    .withColumn("games_played_so_far", sum(col("games_played")).over(windowSpec))\
        .select("player_id","event_date","games_played_so_far")\
            .orderBy("player_id").show()

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+---------+----------+-------------------+
|player_id|event_date|games_played_so_far|
+---------+----------+-------------------+
|        1|2016-03-01|                  5|
|        1|2016-05-02|                 11|
|        1|2017-06-25|                 12|
|        3|2016-03-02|                  5|
|        3|2018-07-03|                 17|
+---------+----------+-------------------+

